In [1]:
import numpy as np
import math
from functools import reduce
import time
import sys
import os
import numpy as np

path_project = "\\".join(os.getcwd().split("\\")[:-1])
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, path_project)
from pathlib import Path
import nidaqmx

def lcm(a, b):
    """Calculate the least common multiple of two numbers."""
    return abs(a * b) // math.gcd(a, b)


def lcm_of_list(numbers):
    """Find the LCM of a list of numbersa."""
    return reduce(lcm, numbers)

def seqtime(seq_tb):
    return np.sum([pulse[-1] for pulse in seq_tb])

# some constants
Hz = 1e-9 # GHz
kHz = 1e-6 # GHz
MHz = 1e-3 # GHz
pi = np.pi


In [2]:
import nidaqmx
from nidaqmx.constants import TerminalConfiguration, VoltageUnits, Edge, AcquisitionType, READ_ALL_AVAILABLE
from nidaqmx.stream_readers import AnalogSingleChannelReader

from hardware import config_custom as hcf
from hardware.hardwaremanager import HardwareManager
from hardware.pulser.pulser import (
    OutputState,
    TriggerStart,
    TriggerRearm,
    HIGH,
    LOW,
    INF,
    REPEAT_INFINITELY
)
timebase = lcm_of_list(
    [hcf.VDISYN_timebase, hcf.SIDIG_timebase, hcf.PS_timebase, hcf.RSRF_timebase]
)

hm = HardwareManager()
# adds synthesizer, laser, and pulse generator 
hm.add_default_hardware()


VDI Sythesizer Serail Port Open
Connect to Pulse Streamer via JSON-RPC.
IP / Hostname: 192.168.1.26
Pulse Streamer 8/2 firmware: v1.7.2
Client software: v1.7.0


## Add the digitizer 

In [3]:
from hardware.daq.sidig import FIFO_DataAcquisition
from hardware.daq.sidig import FIFO_DataAcquisition, DCCOUPLE, TERMIN_INPUT_50OHM, TERMIN_INPUT_1MOHM

SIDIG_ADDRESS = "dev/spcm0"
SIDIG_maxsr = 500e6  # Hz
SIDIG_timebase = int(1 / SIDIG_maxsr * 1e9)  # ns
dig=FIFO_DataAcquisition(
                sn_address=SIDIG_ADDRESS
            )

# Some Parameters 
the parameters that will be used in the measurement and need to be set in the GUI


In [4]:
# ---------------------
# Settings for MW
mw_freq = 398.54667  # GHz
mw_hopspan = 2*3.5E-3 # GHz
# mw_hoptime = 1000.0 # ns

mw_powervolt = 5.0  # voltage 0.0 to 5.0
mw_phasevolt = 0.0  # voltage 0.0 to 5.0

# setting for data acquisition
min_volt = -10.0E-3 # [V]
max_volt = 150.0E-3

# setting for laser
laser_current = 95.0  # percentage

# settings for the RF
rf_freq = 600 # MHz

# -------------------
# setting for the sensing sequence
# # nuclear spin preparation
ninit_pihalf = 1000 #ns

# in a locking block
t_lock_idle = 20 # ns
t_lock_rf = 3980 # ns
t_lock = t_lock_idle + t_lock_rf
n_lock = 2
t_lbloc = t_lock*n_lock

t_lbloc_mwwait = 20.0
t_lbloc_mw = t_lbloc-t_lbloc_mwwait


# in a laser read and init block
t_ribloc = t_lbloc
t_ribloc_wait = 400.0
t_ribloc_isc = 800.0
t_ribloc_laser = t_ribloc - t_ribloc_wait - t_ribloc_isc

# in free evolution
t_fevol = 80e3 # [ns] it is better to be in the multiple of lblock time
n_fevol_div = t_fevol // t_lbloc
t_fevol = n_fevol_div * t_lbloc

# dark reference block
t_drbloc = t_ribloc + t_lbloc
# bright reference block
t_brbloc = t_ribloc

# clearing block using a train of laser pulses, clean the memory effect
t_cbloc = t_fevol - t_brbloc - t_drbloc - t_ribloc
t_cbloc_laser = 10 # ns
t_cbloc_isc = 400 #ns
n_cbloc_laser = int(t_cbloc//(t_cbloc_laser+t_cbloc_isc)//4)
t_cbloc_pad = t_cbloc - n_cbloc_laser*(t_cbloc_laser+t_cbloc_isc)

n_track = 15000 # repetition of the free evolution + the detection block 
t_atrack = t_lbloc + t_fevol
T_alltrack = n_track*t_atrack # [ns] total time = n_track*(free evolution time + detection block time * number of lbloc)

# time_stop = 120
rate_refresh = 10 # 10Hz refreshing rate
# -------------------
idx_pointer = 0
reps = 0
paraset = dict(
    mw_freq = mw_freq,
    # mw_hoptime=mw_hoptime,
    mw_hopspan=mw_hopspan,
    mw_powervolt=mw_powervolt,
    mw_phasevolt=mw_phasevolt,
    min_volt=min_volt,
    max_volt=max_volt,
    laser_current=laser_current,
    rf_freq=rf_freq,
    # pulse sequence------
    ninit_pihalf=ninit_pihalf,
    t_lock_idle = t_lock_idle, 
    t_lock_rf = t_lock_rf, 
    n_lock = n_lock, 
    #--------------
    t_lbloc_mwwait = t_lbloc_mwwait,
    t_lbloc_mw = t_lbloc_mw, 
    t_ribloc = t_ribloc, 
    t_ribloc_wait = t_ribloc_wait,
    t_ribloc_isc = t_ribloc_isc,
    t_ribloc_laser = t_ribloc_laser,
    #---------
    t_fevol=t_fevol,
    t_brbloc=t_brbloc,
    t_drbloc=t_drbloc,
    #-------------
    t_cbloc = t_cbloc,
    t_cbloc_laser = t_cbloc_laser,
    t_cbloc_isc = t_cbloc_isc,
    n_cbloc_laser = n_cbloc_laser,
    t_cbloc_pad = t_cbloc_pad,
    n_track=n_track,
    idx_pointer=idx_pointer,
    reps=reps)


In [5]:
# parameter checking

def check_parameters(paraset):
    assert paraset["mw_powervolt"] <= 5 and paraset["mw_powervolt"]>=0
    assert paraset["mw_phasevolt"] <= 5 and paraset["mw_phasevolt"]>=0
    timeparameters = [
        paraset["t_lbloc_mw"], 
        paraset["t_lbloc_mwwait"],
        paraset["t_lock_idle"],
        paraset["t_lock_rf"],
        paraset["ninit_pihalf"],
        paraset["t_fevol"]
    ]
    for tt in timeparameters:
        assert tt%timebase == 0
    t_lbloc = paraset["n_lock"]*(paraset["t_lock_idle"]+paraset["t_lock_rf"])
    assert paraset["t_fevol"]%t_lbloc == 0
    assert paraset["ninit_pihalf"]*2 <= t_lbloc
    assert paraset["t_fevol"] >= t_lbloc+paraset["t_drbloc"]+paraset["t_brbloc"]+paraset["t_ribloc"]
    assert t_cbloc == paraset["t_cbloc_pad"]+(paraset["t_cbloc_laser"]+paraset["t_cbloc_isc"])*paraset["n_cbloc_laser"]
check_parameters(paraset)

## Pulse Sequence for single-point frequency tracking of ODMR with emulated nuclear radio signals

In [6]:
def seqtime_tb(seq_tb):
    return np.sum([pulse[-1] for pulse in seq_tb])
def seqtime_cb(seq_cb):
    return np.sum([pulse[-0] for pulse in seq_cb])

In [7]:
# --------------------------------------------------------------------------------
# set up the pulse sequence ----------------------------------------------------------
# set the channel general offsets--------------------------------------------------
ps_choffs = {
    "laser": 0,
    "dclk": 0.0,
    "dtrig": 0.0,
    "mwA": 0,
    "mwB": 0,
    "rftrig": 0,
    "Bz": 0,  # AO 0
    "Bx": 0,  # AO 1
}
hm.pg.setChOffset(ps_choffs)
# ------------------------------------------------------------------------------
hm.pg.resetSeq()


t_lbloc = paraset["n_lock"]*(paraset["t_lock_idle"]+paraset["t_lock_rf"])
t_rf_pihalf = paraset["ninit_pihalf"]
n_track = paraset["n_track"]
n_atrack_div = int(t_atrack//t_lbloc)
subseq_ninit = [
    (["rftrig","dtrig"], t_lbloc)
]
t_fevol = paraset["t_fevol"] 
t_atrack = t_fevol+t_lbloc
t_trig = 200
t_ninit = t_lbloc + t_ribloc
t_readat = paraset["t_ribloc_wait"]+900+160

subseq_fevol = [([], t_fevol)]
subseq_ninit_rf = [(t_ninit-t_rf_pihalf-t_ribloc, LOW), (t_rf_pihalf, HIGH), (t_ribloc, LOW)]
subseq_ninit_dtrig = [(t_trig, HIGH), (t_ninit-t_trig, LOW)] 
subseq_ninit_pad = [(t_ninit, LOW)]
n_fevol_div = int(paraset["t_fevol"]//t_lbloc)
n_fevol_div_half = n_fevol_div//2

subseq_cbloc_laser =  [(t_cbloc_laser, HIGH), (t_cbloc_isc, LOW)]*n_cbloc_laser+[(t_cbloc_pad, LOW)]
# subseq_cbloc_laser = [(t_cbloc_laser, HIGH), (t_cbloc_isc, LOW)]*n_cbloc_laser+[(t_cbloc_pad, LOW)]
subseq_ribloc_laser = [(t_ribloc_wait, LOW), (t_ribloc_laser, HIGH), (t_ribloc_isc, LOW)]
subseq_drbloc_laser = [(t_drbloc-t_ribloc, LOW)] + subseq_ribloc_laser
subseq_brbloc_laser = subseq_ribloc_laser

subseq_drbloc_mw = [(t_lbloc_mwwait, LOW), (t_lbloc_mw, HIGH)] + [(t_ribloc, LOW)]
subseq_brbloc_mw = [t_brbloc, LOW]
subseq_cbloc_mw = [(t_cbloc, LOW)]
t_lblochalf = int(t_lbloc/2//timebase)*timebase

seq_laser = subseq_ninit_pad+n_track*(subseq_cbloc_laser+subseq_drbloc_laser+subseq_brbloc_laser+[(t_lbloc, LOW)]+subseq_ribloc_laser)
seq_mwA = subseq_ninit_pad+n_track*([(t_cbloc, LOW)]+subseq_drbloc_mw+[(t_brbloc, LOW)]+[(t_lbloc_mwwait, LOW), (t_lbloc_mw, HIGH)]+[(t_ribloc, LOW)])
# seq_mwA = subseq_init_pad+(subseq_fevol+[(t_lbloc, LOW)]*n_lbloc)*n_track
# seq_dclk = subseq_init_pad + [(t_lbloc-t_lblochalf, LOW), (t_trig, HIGH), (t_lblochalf-t_trig, LOW)]*(n_lbloc+n_fevol_div)*n_track
seq_dclk = subseq_ninit_pad + n_track*[(t_readat, LOW), (t_trig, HIGH),(t_ribloc-t_readat-t_trig, LOW)]*n_atrack_div
seq_dtrig = subseq_ninit_dtrig + [(t_atrack*n_track, LOW)]
seq_rftrig = subseq_ninit_rf + [(t_atrack*n_track, LOW)]


# sdtrig settings 
t_sdtrig=20
t_after_laser_wait=10
subseq_ribloc_sdtrig = [(t_ribloc_wait, LOW), (t_sdtrig, HIGH), (t_ribloc_laser+t_ribloc_isc-t_sdtrig, LOW)]
subseq_cbloc_sdtrig =  [(t_cbloc_laser, LOW), (t_cbloc_isc, LOW)]*n_cbloc_laser+[(t_cbloc_pad, LOW)]
subseq_drbloc_sdtrig = [(t_drbloc-t_ribloc, LOW)] + subseq_ribloc_sdtrig
subseq_brbloc_sdtrig = [(t_ribloc_wait, LOW), (t_sdtrig, HIGH), (t_ribloc_laser+t_ribloc_isc-t_sdtrig-t_sdtrig-t_after_laser_wait, LOW),(t_sdtrig, HIGH), (10, LOW)]

# subseq_ribloc_sdtrig = [(t_ribloc_wait, LOW), (t_sdtrig, LOW), (t_ribloc_laser+t_ribloc_isc-t_sdtrig, LOW)]
# temp= [(t_ribloc_wait, LOW), (t_sdtrig, HIGH), (t_ribloc_laser+t_ribloc_isc-t_sdtrig, LOW)]
# subseq_cbloc_sdtrig =  [(t_cbloc_laser, LOW), (t_cbloc_isc, LOW)]*n_cbloc_laser+[(t_cbloc_pad, LOW)]
# subseq_drbloc_sdtrig = [(t_drbloc-t_ribloc, LOW)] + subseq_ribloc_sdtrig
# subseq_brbloc_sdtrig = [(t_ribloc_wait, LOW), (t_sdtrig, LOW), (t_ribloc_laser+t_ribloc_isc-t_after_laser_wait, LOW),(t_sdtrig, LOW), (10, LOW)]
seq_sdtrig = subseq_ninit_pad+n_track*(subseq_cbloc_sdtrig+subseq_drbloc_sdtrig+subseq_brbloc_sdtrig+[(t_lbloc, LOW)]+subseq_ribloc_sdtrig)


In [8]:
import pickle

In [9]:
def read(time_stop):
    
    hm.pg.setDigital("laser", seq_laser, offset=True)
    hm.pg.setDigital("mwA", seq_mwA, offset=True)
    # seq_mwB = seq_mwA
    # hm.pg.setDigital("mwB", seq_mwB, offset=True)
    # hm.pg.setDigital("dclk", seq_dclk, offset=True)
    # hm.pg.setDigital("dtrig", seq_dtrig, offset=True)
    hm.pg.setDigital("rftrig", seq_rftrig, offset=True)

    hm.pg.setDigital("sdtrig",seq_sdtrig, offset=True)
    # to generate fake signal for testing
    amp_a = 0.0
    amp_b = 1.0
    amp_c = 0.0
    omega = 2 * pi * 15 * Hz
    omega_b = 2 * pi * 32.425266 * Hz
    omega_c = 2 * pi * 25 * Hz
    decay_a = 0.2*Hz
    decay_b = 2*Hz
    decay_c = 0.2*Hz
    rfomega = 2 * pi * 80 * kHz
    dt_inlock = 199.0
    t_lock_idle = paraset["t_lock_idle"]
    t_lock_rf = paraset["t_lock_rf"]
    assert t_lock_rf%dt_inlock==0

    tarray_inlock = np.arange(0, t_lock_rf
                            , dt_inlock)
    bzlevel0 = np.sin(rfomega*tarray_inlock)
    bzlevel0_flip = np.flip(bzlevel0)
    i_track = 0
    wf_Bz_xpc = []
    wf_Bz_xpc += [(t_ninit-t_ribloc, LOW)]
    wf_Bz_ypc = []
    wf_Bz_ypc += subseq_ninit_pad
    # randphase_a = np.random.random(1)*pi
    # randphase_b = np.random.random(1)*pi
    randphase_a = 0.0
    randphase_b = 0.0
    randphase_c = 0.0
    bzevol_cos = np.zeros(n_track)
    for it in range(n_track):
        t_evol = t_ninit + t_atrack*it + t_fevol + t_lock_idle
        factor_evol_cos = float(amp_a*np.cos(omega*t_evol+randphase_a)*np.exp(-decay_a*t_evol)+\
                                amp_b*np.cos(omega_b*t_evol+randphase_b)*np.exp(-decay_b*t_evol)+\
                                amp_c*np.cos(omega_c*t_evol+randphase_c)*np.exp(-decay_c*t_evol))
                                
        factor_evol_cos /= (amp_a+amp_b+amp_c)
        # factor_evol_cos *= 0.1
        bzevol_cos[it] = factor_evol_cos
        # factor_evol_sin = amp_a*np.sin(omega*(t_evol+t_lbloc+t_lbloc_idle)+randphase_a)+amp_b*np.sin(omega_b*(t_evol+t_lbloc+t_lbloc_idle)+randphase_b)
        # factor_evol_sin /= (amp_a+amp_b)
        bzlevel_xpc = bzlevel0*factor_evol_cos # X phase cycle
        bzlevel_xpc_flip = bzlevel0_flip*factor_evol_cos
        # bzlevel_ypc = bzlevel0*factor_evol_sin # Y phase cycle

        # comment out if trying smooth radio signal---------
        # bzlevel_ypc_flip = bzlevel0_flip*factor_evol_sin
        wf_bz_inlock_xpc = [(t_lock_idle, 0)]+[(dt_inlock, bzl) for bzl in bzlevel_xpc]
        wf_bz_inlock_xpc +=  [(t_lock_idle, 0)]+[(dt_inlock, bzl) for bzl in bzlevel_xpc_flip]
        # wf_bz_inlock_ypc = [(t_lbloc_idle, 0)]+[(dt_inlock, bzl) for bzl in bzlevel_ypc]
        # wf_bz_inlock_ypc +=  [(t_lbloc_idle, 0)]+[(dt_inlock, bzl) for bzl in bzlevel_ypc_flip]
        # # wf_bz_inlock = [(t_lbloc, 1)]
        wf_Bz_xpc += [(t_fevol, 0)]+wf_bz_inlock_xpc
        # # wf_Bz_ypc += [(t_fevol, 0)]+wf_bz_inlock_ypc
        # ----------------------------------------------------------
        # wf_Bz_xpc += [(t_atrack, factor_evol_cos)]
        # wf_Bz_ypc += [(t_fevol, 0)]+wf_bz_inlock_ypc
    wf_Bz_xpc += [(t_ribloc, LOW)]

    hm.pg.setAnalog("Bz", wf_Bz_xpc, offset=True)
    # hm.pg.setAnalog("Bx", wf_Bz_xpc, offset=True)
    assert seqtime_cb(seq_laser) == seqtime_cb(seq_mwA) == seqtime_cb(wf_Bz_xpc) == seqtime_cb(seq_dclk)
    hm.pg.setTrigger(TriggerStart.SOFTWARE, rearm=TriggerRearm.AUTO)
    hm.pg.stream(n_runs=REPEAT_INFINITELY)
    ## set up the data aquication with the digitizer 
    # min_volt = -0.002 # [V]
    # max_volt = 0.010 # [V]

    t_seq = seqtime_cb(seq_laser)
    rate_refresh = .01 # Hz rate of refreshing the data streaming
    amp_input = 1000
    readout_ch = hcf.SIDIG_chmap["apd"]
    num_segment = int(2/rate_refresh*1e9/t_seq)//32*32

    # configures the readout to match the pulse sequence  
    pretrig_size=int((t_ribloc_wait/2)*hcf.SIDIG_maxsr/1E9)//64*64 # pretrigger based on the t_wait time
    # posttrig_size=int((t_ribloc+t_ribloc_isc)*hcf.SIDIG_maxsr/1E9)//64*64 # posttrigger based on the t_laser time and t_isc
    posttrig_size=int((t_ribloc//4)*hcf.SIDIG_maxsr/1E9)//64*64 # posttrigger based on the t_laser time and t_isc

    segment_size = pretrig_size+posttrig_size 
    segment_size = 2**int(np.log2(segment_size)+1) # make it power of 2
    posttrig_size = segment_size-pretrig_size # recalculate posttrigger size to ensure it is power of 2
    # To set the configuration, make a dictionary with the key and value
    dig.reset_param()
    dig.assign_param(dict(
                readout_ch=readout_ch, 
                amp_input=amp_input, 
                num_segment=num_segment,
                pretrig_size=pretrig_size,
                posttrig_size=posttrig_size,
                segment_size=segment_size,
                terminate_input=TERMIN_INPUT_1MOHM,
                couple_input=1
                ))

    # -----------------------------------------------------------------------
    # set the MW frequency --------------------------------------------------
    try:
        hm.mwsyn.open()
    except Exception as ee:
        print(ee)
    mw_freq = paraset["mw_freq"] + paraset["mw_hopspan"]/2.0
    errorbyte, freq_actual = hm.mwsyn.cw_frequency(mw_freq / hcf.VDISYN_multiplier)
    print(f"CW Freqeuncy Setting Sent:{mw_freq/hcf.VDISYN_multiplier} GHz")
    print(f"Actual Output CW Freqeuncy :{freq_actual} GHz")
    # -----------------------------------------------------------------------
    # set MW power and phase using NIIO ------------------------------------
    mwpower_vlevel = paraset["mw_powervolt"]  # 5V equals to max power
    task_uca = nidaqmx.Task("UCA")  # user controlled attenuation
    task_uca.ao_channels.add_ao_voltage_chan(hcf.NI_ch_UCA, min_val=0, max_val=10)
    # task_uca.timing.cfg_samp_clk_timing(hcf.NI_sampling_max/100.0, sample_mode=AcquisitionType.CONTINUOUS)
    task_uca.start()
    task_uca.write([mwpower_vlevel], auto_start=False)

    mwphase_vlevel = paraset["mw_phasevolt"]  # voltage to phase shifter
    task_mwbp = nidaqmx.Task("MW B Phase")  # user controlled attenuation
    task_mwbp.ao_channels.add_ao_voltage_chan(hcf.NI_ch_MWBP, min_val=0, max_val=10)
    # task_uca.timing.cfg_samp_clk_timing(hcf.NI_sampling_max/100.0, sample_mode=AcquisitionType.CONTINUOUS)
    task_mwbp.start()
    task_mwbp.write([mwphase_vlevel], auto_start=False)
    # -----------------------------------------------------------------------
    # set up laser----------------------------------------------------------
    current_percent = paraset["laser_current"]
    hm.laser.laser_off()
    hm.laser.set_analog_control_mode("current")
    hm.laser.set_modulation_state("Pulsed")
    hm.laser.set_diode_current(current_percent, save_memory=False)
    hm.laser.laser_on() # turn on laser
    dig.set_config() # set the digitizer configuration 
    dig.start_buffer() # get ready to acquire data 
    hm.pg.startNow()

    data_buffer_drbloc = np.zeros((n_track,pretrig_size+posttrig_size))
    data_buffer_brbloc = np.zeros((n_track,pretrig_size+posttrig_size))
    data_buffer_lbloc = np.zeros((n_track,pretrig_size+posttrig_size))
    data_buffer_ribloc = np.zeros((n_track,pretrig_size+posttrig_size))


    idx_pointer = 0
    reps=0
    start_time = time.time()
    while time.time() - start_time < time_stop:
        rawraw = dig.stream()

        if rawraw is not None:
            num_segs = np.shape(rawraw)[0]
            # assertion ensures the data is multiple of 4 for 4 readout block strucutre, 
            # drbloc, brbloc, lbloc, ribloc
            assert num_segs % 4 == 0

            # seperate data into respective readout blocks
            rawraw_drbloc = rawraw[0::4, :, :]
            rawraw_brbloc = rawraw[1::4, :, :]
            rawraw_lbloc = rawraw[2::4, :, :]
            rawraw_ribloc = rawraw[3::4, :, :]

            
            idx_track = num_segs//4 # number of repititions of each data block type in pushed raw data     
            for i in range(idx_track):
                idx=(idx_pointer+i)%n_track
                data_buffer_brbloc[idx,:]+=np.ravel(rawraw_brbloc[i])
                data_buffer_drbloc[idx,:]+=np.ravel(rawraw_drbloc[i])
                data_buffer_lbloc[idx,:]+=np.ravel(rawraw_lbloc[i])
                data_buffer_ribloc[idx,:]+=np.ravel(rawraw_ribloc[i])
                # print(idx)
            idx_pointer+=idx_track
            if idx_pointer >= n_track:
                reps+=1
            idx_pointer = idx_pointer%n_track  

    hm.pg.forceFinal()
    hm.pg.constant(OutputState.ZERO())
    hm.pg.reset()
    hm.laser.laser_off()
    task_mwbp.close()
    task_uca.close()
    hm.mwsyn.close_gracefully()
    dig.stop_card()

    time.sleep(1)
    paraset['idx_pointer']=idx_pointer
    paraset["reps"] = reps
    buffer_data=[data_buffer_drbloc,data_buffer_brbloc,data_buffer_lbloc,data_buffer_ribloc]
    # Define the file name
    fname = f"output/20250207_odmr_tracking_snr/data_dev_pulsed_odmr_tracking_changedetection_{time.time()}_runlen_{time_stop}.pkl"

    # Assuming 'buffer_data' and 'paraset' are defined
    data_to_save = {
        "buffer_data": buffer_data,
        "paraset": paraset
    }

    # Save the data to a pickle file
    with open(fname, "wb") as file:
        pickle.dump(data_to_save, file)

    print(f"Data successfully saved to {fname}")

            

    return(fname)


### Data Processing 

In [10]:
# [  1.   1.   1.   5.  10.  20.  20.  30.  40.  50.  60.  70.  80.  90.
#  100. 110. 120. 130. 140. 150. 170. 180. 190. 220. 240. 300. 350. 400.
#  450. 500. 600.]


In [11]:
read(240)

Setting Analog channel 0
CW Freqeuncy Setting Sent:16.606257083333333 GHz
Actual Output CW Freqeuncy :16.60625708475709 GHz
SETTINGS: card timeout =  5 s
SETTINGS: # of segments =  2048 Sa
SETTINGS: sampling freq =  0.5 GHz
SETTINGS: pretrig size =  64 Sa
SETTINGS: termination =  0
notify size:  65536.0 Sa
VDI Synthesizer Serial port closed.
Card stopped
Data successfully saved to output/20250207_odmr_tracking_snr/data_dev_pulsed_odmr_tracking_changedetection_1739571768.556554_runlen_240.pkl


'output/20250207_odmr_tracking_snr/data_dev_pulsed_odmr_tracking_changedetection_1739571768.556554_runlen_240.pkl'

In [20]:
import pickle

# Specify the file name (replace with the actual filename if necessary)
fname = ".\output/20250207_odmr_tracking_snr/data_dev_pulsed_odmr_tracking_changedetection_1739571768.556554_runlen_240.pkl"

try:
    # Open and read the pickle file
    with open(fname, "rb") as file:
        loaded_data = pickle.load(file)

    # Extract the contents
    buffer_data = loaded_data["buffer_data"]
    paraset = loaded_data["paraset"]

    print("Data successfully loaded:")
    print("Buffer Data:", buffer_data)
    print("Parameter Set:", paraset)

except FileNotFoundError:
    print(f"Error: File {fname} not found.")
except pickle.UnpicklingError:
    print("Error: Failed to load pickle file.")


Data successfully loaded:
Buffer Data: [array([[-8.01721191, -8.02001953, -8.02770996, ..., 27.26623535,
        27.24621582, 27.24768066],
       [-8.13183594, -8.12463379, -8.1270752 , ..., 26.89025879,
        26.90161133, 26.88513184],
       [-8.11462402, -8.11462402, -8.10681152, ..., 27.0723877 ,
        27.06311035, 27.07141113],
       ...,
       [-8.2947998 , -8.28747559, -8.2767334 , ..., 27.01660156,
        27.01635742, 27.0177002 ],
       [-8.23730469, -8.22949219, -8.23962402, ..., 26.97680664,
        26.97912598, 26.98852539],
       [-8.22253418, -8.22583008, -8.24194336, ..., 26.81530762,
        26.80053711, 26.78442383]]), array([[-8.27038574, -8.26647949, -8.25244141, ..., 26.88513184,
        26.90283203, 26.91674805],
       [-8.26318359, -8.25476074, -8.2532959 , ..., 26.72485352,
        26.72070312, 26.70141602],
       [-8.30163574, -8.30419922, -8.3059082 , ..., 26.73657227,
        26.71228027, 26.70629883],
       ...,
       [-8.18066406, -8.18347168, 

In [21]:
data_buffer_drbloc = buffer_data[0]
data_buffer_brbloc = buffer_data[1]
data_buffer_lbloc = buffer_data[2]
data_buffer_ribloc = buffer_data[3]
idx_pointer=paraset['idx_pointer']
reps=paraset["reps"]

In [22]:
def average_repeated_data(arr, start, stop,index,reps):
    arr[0:index,:]/=(reps+1)
    arr[index:]/=reps
    sliced_data=arr[:,start:stop]
    averaged=np.mean(sliced_data, axis=1)
    return(averaged)

In [23]:
buffer_brbloc = average_repeated_data(data_buffer_brbloc, 0,518, idx_pointer,reps)
buffer_drbloc = average_repeated_data(data_buffer_drbloc,  0,518, idx_pointer,reps)
buffer_lbloc = average_repeated_data(data_buffer_lbloc,  0,518, idx_pointer,reps)
buffer_ribloc = average_repeated_data(data_buffer_ribloc, 0,518, idx_pointer,reps)

In [24]:
# import plotly.graph_objects as go

# fig = go.Figure()

# # Add traces for each data buffer
# # # fig.add_trace(go.Scatter(y=test4, mode='lines', name='DRB Loc'))
# timetime = t_atrack * np.arange(0, n_track, 1) / 1E9
# fig.add_trace(go.Scatter(x=timetime, y=buffer_drbloc, mode='lines', name='DRB Loc'))
# fig.add_trace(go.Scatter(x=timetime, y=buffer_brbloc, mode='lines', name='BRB Loc'))
# fig.add_trace(go.Scatter(x=timetime, y=buffer_lbloc, mode='lines', name='LB Loc'))
# fig.add_trace(go.Scatter(x=timetime, y=buffer_ribloc, mode='lines', name='RIB Loc'))


# # Show the plot
# fig.show()


In [25]:
# import matplotlib.pyplot as plt
# import numpy as np

# timetime = t_atrack * np.arange(0, n_track, 1) / 1E9


# # Create subplots with two columns
# fig, axes = plt.subplots(2, 1, figsize=(8, 6))
# name = ["dark", "bright", "bg", "signal"]
# # Plot the original data (time domain)

# axes[0].plot(timetime,(buffer_drbloc-buffer_lbloc)*1E3, label=name[0])

# axes[0].plot(timetime,(buffer_brbloc-buffer_lbloc)*1E3, label=name[1])

# # axes[0].plot(timetime,(buffer_lbloc)*1E3, label=name[2])

# axes[0].plot(timetime,(buffer_ribloc-buffer_lbloc)*1E3, label=name[3])


# # axes[0].set_xlabel("Time [s]")
# axes[0].set_ylabel("PL [mV]")
# axes[0].set_title("Original Signal")
# axes[0].legend()
# axes[0].grid()



# freq = np.fft.fftfreq(len(buffer_ribloc), t_atrack / 1E9)
# sig_fft = np.abs(np.fft.fft(buffer_ribloc))
# positive_freqs = freq > 0
# axes[1].plot(freq[positive_freqs], sig_fft[positive_freqs], label=f"FFT {name[3]}")


# freq = np.fft.fftfreq(len(buffer_drbloc), t_atrack / 1E9)
# sig_fft = np.abs(np.fft.fft(buffer_drbloc))
# positive_freqs = freq > 0
# axes[1].plot(freq[positive_freqs], sig_fft[positive_freqs], label=f"FFT {name[0]}")


# freq = np.fft.fftfreq(len(buffer_brbloc), t_atrack / 1E9)
# sig_fft = np.abs(np.fft.fft(buffer_brbloc))
# positive_freqs = freq > 0
# axes[1].plot(freq[positive_freqs], sig_fft[positive_freqs], label=f"FFT {name[1]}")


# freq = np.fft.fftfreq(len(buffer_lbloc), t_atrack / 1E9)
# sig_fft = np.abs(np.fft.fft(buffer_lbloc))
# positive_freqs = freq > 0
# axes[1].plot(freq[positive_freqs], sig_fft[positive_freqs], label=f"FFT {name[0]}")


# axes[1].set_xlabel("Frequency [Hz]")
# axes[1].set_ylabel("Amplitude")
# axes[1].set_title("FFT of Signals (Positive Frequencies)")
# axes[1].legend()
# axes[1].grid()
# #  # Adjust layout and show the plots
# plt.tight_layout()
# plt.show()


In [26]:
# import plotly.graph_objects as go
# import numpy as np

# # Generate time data
# timetime = t_atrack * np.arange(0, n_track, 1) / 1E9
# name = ["dark", "bright", "bg", "signal"]

# # Create time domain plot
# fig_time = go.Figure()
# # fig_time.add_trace(go.Scatter(x=timetime, y=(buffer_drbloc-buffer_lbloc)*1E3, mode='lines', name=name[0]))
# # fig_time.add_trace(go.Scatter(x=timetime, y=(buffer_brbloc-buffer_lbloc)*1E3, mode='lines', name=name[1]))
# # # fig_time.add_trace(go.Scatter(x=timetime, y=(buffer_lbloc)*1E3, mode='lines', name=name[2]))
# # fig_time.add_trace(go.Scatter(x=timetime, y=(buffer_ribloc-buffer_lbloc)*1E3, mode='lines', name=name[3]))
# fig_time.add_trace(go.Scatter(x=timetime, y=(buffer_drbloc-buffer_lbloc)*1E3, mode='lines', name=name[0]))
# fig_time.add_trace(go.Scatter(x=timetime, y=(buffer_brbloc-buffer_lbloc)*1E3, mode='lines', name=name[1]))
# # fig_time.add_trace(go.Scatter(x=timetime, y=(buffer_lbloc)*1E3, mode='lines', name=name[2]))
# fig_time.add_trace(go.Scatter(x=timetime, y=(buffer_ribloc-buffer_lbloc)*1E3, mode='lines', name=name[3]))

# fig_time.update_layout(title='Original Signal',
#                        yaxis_title='PL [mV]',
#                        xaxis_title='Time [s]')

# # Compute and plot FFT
# fig_fft = go.Figure()

# def add_fft_trace(buffer, label):
#     # buffer=buffer[5:-5]
#     freq = np.fft.fftfreq(len(buffer), (t_atrack / 1E9))
#     sig_fft = np.abs(np.fft.fft(buffer))
#     # bg_fft = np.abs(np.fft.fft(buffer_lbloc[100:-100]))
#     positive_freqs = freq > 0
#     fig_fft.add_trace(go.Scatter(x=freq[positive_freqs], y=(sig_fft)[positive_freqs], mode='lines', name=f"FFT {label}"))

# add_fft_trace(buffer_ribloc, name[3])
# add_fft_trace(buffer_drbloc, name[0])
# add_fft_trace(buffer_brbloc, name[1])
# add_fft_trace(buffer_lbloc, name[2])

# fig_fft.update_layout(title='FFT of Signals (Positive Frequencies)',
#                       xaxis_title='Frequency [Hz]',
#                       yaxis_title='Amplitude')
# # Show plots
# fig_time.show()
# fig_fft.show()


In [ ]:
name = ["dark", "bright", "bg", "signal"]
import plotly.graph_objects as go
fig_fft = go.Figure()

def add_fft_trace(buffer, label):
    # buffer=buffer[5:-5]
    freq = np.fft.fftfreq(len(buffer), (t_atrack / 1E9))
    sig_fft = np.abs(np.fft.fft(buffer))
    sig_fft -= np.abs(np.fft.fft(buffer_drbloc))
    # bg_fft = np.abs(np.fft.fft(buffer_lbloc[100:-100]))
    positive_freqs = freq > 0
    fig_fft.add_trace(go.Scatter(x=freq[positive_freqs], y=(sig_fft)[positive_freqs], mode='lines', name=f"FFT {label}"))

add_fft_trace(buffer_ribloc, name[3])
add_fft_trace(buffer_drbloc, name[0])
add_fft_trace(buffer_brbloc, name[1])
add_fft_trace(buffer_lbloc, name[2])

fig_fft.update_layout(title='FFT of Signals (Positive Frequencies)',
                      xaxis_title='Frequency [Hz]',
                      yaxis_title='Amplitude')
# Show plots
fig_fft.show()


: 